In [20]:
# Импортируем нужные библиотеки
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
pwd # рабочая папка

'c:\\Users\\vvtom\\YandexDisk\\Prog\\ITNordic\\Final_project'

In [22]:
# список входящих файлов
import os

for dirname, _, filenames in os.walk('c:\\Users\\vvtom\\YandexDisk\\Prog\\ITNordic\\Final_project\\input_data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

c:\Users\vvtom\YandexDisk\Prog\ITNordic\Final_project\input_data\events.csv
c:\Users\vvtom\YandexDisk\Prog\ITNordic\Final_project\input_data\item_properties_part1.csv
c:\Users\vvtom\YandexDisk\Prog\ITNordic\Final_project\input_data\item_properties_part2.csv


In [23]:
# загружаем данные 
events_df = pd.read_csv('input_data\events.csv')
print(events_df.info())
print(events_df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 105.1+ MB
None
          timestamp     visitorid        itemid  transactionid
count  2.756101e+06  2.756101e+06  2.756101e+06   22457.000000
mean   1.436424e+12  7.019229e+05  2.349225e+05    8826.497796
std    3.366312e+09  4.056875e+05  1.341954e+05    5098.996290
min    1.430622e+12  0.000000e+00  3.000000e+00       0.000000
25%    1.433478e+12  3.505660e+05  1.181200e+05    4411.000000
50%    1.436453e+12  7.020600e+05  2.360670e+05    8813.000000
75%    1.439225e+12  1.053437e+06  3.507150e+05   13224.000000
max    1.442545e+12  1.407579e+06  4.668670e+05   17671.000000


In [24]:
# Можно для скорости отфильтровать данные по датам через timestamp.
# Однако если мы изначально не знаем какие периоды у нас в данных это выглядит проблематично

start_date, end_date = '2015.06.05', '2015.06.12'
def date_filter_timestamp(df, start_date, end_date):
    start_date = datetime.strptime(start_date, "%Y.%m.%d")
    end_date = datetime.strptime(end_date, "%Y.%m.%d") + timedelta(days=1)
    start_timestamp = datetime.timestamp(start_date) * 1000
    end_timestamp = datetime.timestamp(end_date) * 1000
    filter_data_events = df[(df.timestamp > start_timestamp)&(df.timestamp < end_timestamp)]
    return filter_data_events

# filter_events_df = date_filter_timestamp(events_df, start_date, end_date)


In [25]:
# Предбработка ДФ 
def preproc(df):
    '''Preprocessing func: convert timestamp to datetime, drop column timestamp'''
    df['datetime'] = df.timestamp.apply(lambda ts: datetime.fromtimestamp(ts/1000)) # миллисекунды переводим в секунды
    df = df.drop('timestamp', axis=1)
    df = df.sort_values('datetime')
    df['diffdate'] = df.groupby('visitorid')['datetime'].diff(1)
    df['shiftdate'] = df.groupby('visitorid')['datetime'].shift(1)

    return df

preproc_events_df = preproc(events_df)

In [26]:
preproc_events_df.datetime.describe()

count                          2756101
mean     2015-07-09 09:48:08.348101888
min         2015-05-03 06:00:04.384000
25%         2015-06-05 07:23:14.792000
50%      2015-07-09 17:43:33.599000064
75%         2015-08-10 19:45:05.168000
max         2015-09-18 05:59:47.788000
Name: datetime, dtype: object

In [27]:
# Можно указать фильтр по датам (начало, окончание) для дальнейшей обработки данных
start_date, end_date = '2015.06.05', '2015.06.12'

def date_filter(df, start_date, end_date): # start_date и end_date в формате "%Y.%m.%d"
    '''Фильтр по времени'''
    start_date = datetime.strptime(start_date, "%Y.%m.%d")
    end_date = datetime.strptime(end_date, "%Y.%m.%d") + timedelta(days=1)

    filter_data_events = df[(df.datetime > start_date)&(df.datetime < end_date)]
    return filter_data_events

filter_events_df = date_filter(preproc_events_df, start_date, end_date)


In [28]:
# Указываем продолжительность контрольного периода времени между действиями
control_period = timedelta(minutes=15)

In [29]:
# Данные столбца diffdate промежутки больше control_period и NAT приравниваем к timedelta(0)

filter_events_df['diffdate'][filter_events_df.diffdate > control_period] = timedelta(0)
filter_events_df['diffdate'][filter_events_df.diffdate.isna()] = timedelta(0)
filter_events_df

C:\Users\vvtom\AppData\Local\Temp\ipykernel_15424\693943181.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_events_df['diffdate'][filter_events_df.diffdate > control_period] = timedelta(0)
C:\Users\vvtom\AppData\Local\Temp\ipykernel_15424\693943181.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_events_df['diffdate'][filter_events_df.diffdate.isna()] = timedelta(0)


,visitorid,event,itemid,transactionid,datetime,diffdate,shiftdate
70846,293226,view,390199,NaN,2015-06-05 00:00:01.648,0 days 00:01:22.397000,2015-06-04 23:58:39.251
86166,602506,view,16813,NaN,2015-06-05 00:00:07.368,0 days 00:00:00,NaT
74774,1352054,view,205777,NaN,2015-06-05 00:00:09.444,0 days 00:03:54.302000,2015-06-04 23:56:15.142
74656,346597,view,409344,NaN,2015-06-05 00:00:10.133,0 days 00:00:25.483000,2015-06-04 23:59:44.650
78398,638935,view,329123,NaN,2015-06-05 00:00:12.270,0 days 00:00:00,2015-05-18 21:26:45.113
...,...,...,...,...,...,...,...
253896,205881,view,95974,NaN,2015-06-12 23:59:48.895,0 days 00:00:00,NaT
251157,752543,view,190885,NaN,2015-06-12 23:59:51.271,0 days 00:00:00,NaT
248643,810408,view,114695,NaN,2015-06-12 23:59:51.994,0 days 00:00:00,NaT
248650,664683,view,286592,NaN,2015-06-12 23:59:53.961,0 days 00:00:00,NaT


In [30]:
# Создаем датасет RFD_data с нужными для анализа столбцами

temp = ['visitorid', 'itemid', 'transactionid', 'datetime', 'diffdate']
RFD_data = filter_events_df[temp]

# Оставляем пользователей, у которых есть хотя бы одна покупка
RFD_data['transaction_datetime'] = RFD_data.datetime.where(RFD_data.transactionid.notna())
users_with_transactions = RFD_data.visitorid.where(RFD_data.transactionid.notna()).dropna().unique()
RFD_data = RFD_data[RFD_data.visitorid.isin(users_with_transactions)]

C:\Users\vvtom\AppData\Local\Temp\ipykernel_15424\3842965369.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RFD_data['transaction_datetime'] = RFD_data.datetime.where(RFD_data.transactionid.notna())


In [31]:
RFD_data

,visitorid,itemid,transactionid,datetime,diffdate,transaction_datetime
70952,438441,416170,NaN,2015-06-05 00:01:04.702,0 days 00:05:04.285000,NaT
90071,438441,190606,NaN,2015-06-05 00:01:34.867,0 days 00:00:30.165000,NaT
78462,1170743,164344,11112.0,2015-06-05 00:02:02.703,0 days 00:06:52.863000,2015-06-05 00:02:02.703
70754,892355,370064,NaN,2015-06-05 00:02:31.879,0 days 00:03:15.246000,NaT
86418,503857,135840,NaN,2015-06-05 00:02:42.383,0 days 00:04:44.971000,NaT
...,...,...,...,...,...,...
248787,1204946,206241,NaN,2015-06-12 23:52:16.536,0 days 00:00:00,NaT
254628,531661,32971,NaN,2015-06-12 23:54:21.006,0 days 00:00:00,NaT
254831,531661,102000,NaN,2015-06-12 23:54:50.653,0 days 00:00:29.647000,NaT
258842,1204946,206241,NaN,2015-06-12 23:56:13.806,0 days 00:03:57.270000,NaT


In [32]:
# Создаем переменную now - последняя(максимальная) дата в датасете для расчета времени с последней покупки
now = RFD_data.datetime.max()

# Создаем таблицу RFD_table
RFD_table = RFD_data.groupby('visitorid').agg({'transaction_datetime': lambda x: (now - x.max()).days, # Recency
                                               'transactionid': lambda x: len(x.unique()), # Frequency
                                               'diffdate': sum # Duration
                                            })
# RFD_table.transaction_datetime = RFD_table.transaction_datetime.astype(int)

# Переименовываем столбцы для простоты и лаконичности
RFD_table.rename(columns={'transaction_datetime': 'R',
                          'transactionid': 'F',
                          'diffdate': 'D'}, inplace=True)

RFD_table.sort_values(by='F', ascending=False)

,R,F,D
visitorid,,,
841939,2,31,0 days 10:51:55.149000
138131,1,30,0 days 08:51:12.298000
1093035,1,29,0 days 11:17:12.261000
1161163,0,28,0 days 11:41:53.795000
76757,0,23,0 days 11:21:47.529000
...,...,...,...
534217,1,2,0 days 00:27:01.576000
534712,2,2,0 days 01:34:12.828000
535618,1,2,0 days 00:15:05.794000


In [33]:
# Ранжируем покупателей и ставим оценки в зависимости от возможности разделения по квантилям
RFD_table['R_rank'] = pd.qcut(x=RFD_table.R, q=3, labels=[3, 2, 1])
RFD_table['F_rank'] = pd.qcut(x=RFD_table.F, q=2, labels=[1,2])
RFD_table['D_rank'] = pd.qcut(x=RFD_table.D, q=3, labels=[1,2,3])
RFD_table

,R,F,D,R_rank,F_rank,D_rank
visitorid,,,,,,
3847,1,2,0 days 00:03:26.963000,3,1,1
4241,1,2,0 days 00:01:10.920000,3,1,1
5117,3,2,0 days 00:25:41.807000,2,1,3
11333,3,2,0 days 00:24:56.206000,2,1,3
16158,4,2,0 days 00:07:50.218000,2,1,2
...,...,...,...,...,...,...
1395774,4,3,0 days 00:13:07.001000,2,2,2
1397020,4,2,0 days 00:01:14.657000,2,1,1
1398811,6,2,0 days 00:41:09.648000,1,1,3


In [34]:
# Суммируем оценки R, F, D  

RFD_table[['R_rank','F_rank','D_rank']] = RFD_table[['R_rank','F_rank','D_rank']].astype(int)

RFD_table['RFD'] = RFD_table['R_rank'] + RFD_table['F_rank'] + RFD_table['D_rank']

In [35]:
# Для простоты ранжирования пользователей создаем ф-цию соотнесения пользователя по 3 категориям: Холодный, Теплый, Горячий

def user_rating(rfd):
    if rfd < 5:
        return 'Холодный'
    elif rfd < 7 and rfd >= 5:
        return 'Теплый'
    else:
        return 'Горячий'

RFD_table['U_category'] = RFD_table['RFD'].apply(lambda x: user_rating(x))
# RFD_table[RFD_table.U_category == 'Теплый'].sort_values(by=['RFD'], ascending=False)

# RFD_table[RFD_table.RFD == 4]

In [36]:
RFD_table

,R,F,D,R_rank,F_rank,D_rank,RFD,U_category
visitorid,,,,,,,,
3847,1,2,0 days 00:03:26.963000,3,1,1,5,Теплый
4241,1,2,0 days 00:01:10.920000,3,1,1,5,Теплый
5117,3,2,0 days 00:25:41.807000,2,1,3,6,Теплый
11333,3,2,0 days 00:24:56.206000,2,1,3,6,Теплый
16158,4,2,0 days 00:07:50.218000,2,1,2,5,Теплый
...,...,...,...,...,...,...,...,...
1395774,4,3,0 days 00:13:07.001000,2,2,2,6,Теплый
1397020,4,2,0 days 00:01:14.657000,2,1,1,4,Холодный
1398811,6,2,0 days 00:41:09.648000,1,1,3,5,Теплый
